## Задача 4-1. Линейное программирование: формулирование задачи.

В этой задаче Вам предлагается закодировать задачу TSP для заданного графа в виде задачи ЦЛП с помощью условий MTZ (Миллера—Таккера—Землина) и поработать с библиотекой [PuLP](https://pypi.python.org/pypi/PuLP/1.6.5). Если Вы используете дистрибутив Anaconda, то эта библиотека не находится утилитой conda, зато совершенно нормально устанавливается с помощью pip:

`pip install pulp`

Вам дана функция `dist15`, которая по двум номерам вершин возвращает вес ребра между ними. Номера от 1 до 15, пример взят [отсюда](https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html).
Нужно построить соответствующую задачу ЦЛП и решить её средствами PuLP, чтобы найти оптимальный гамильтонов цикл в графе. Сделать это следует в функции `solve_tsp_with_lp`, которая получает на вход размерность задачи и весовую функцию, а на выходе даёт перестановку номеров вершин графа (нумеруемых с единицы), соответствующую оптимальному гамильтонову циклу.

In [4]:
pip3 install pulp

SyntaxError: invalid syntax (<ipython-input-4-deec51836a20>, line 1)

In [5]:
import pulp

In [6]:
def dist15(i: int, j: int) -> int:
    return (
        list(
            map(
                int, 
                filter(
                    lambda s: len(s.strip()) > 0, '''
                    0 29 82 46 68 52 72 42 51 55 29 74 23 72 46 
                    29  0 55 46 42 43 43 23 23 31 41 51 11 52 21 
                    82 55  0 68 46 55 23 43 41 29 79 21 64 31 51 
                    46 46 68  0 82 15 72 31 62 42 21 51 51 43 64 
                    68 42 46 82  0 74 23 52 21 46 82 58 46 65 23 
                    52 43 55 15 74  0 61 23 55 31 33 37 51 29 59 
                    72 43 23 72 23 61  0 42 23 31 77 37 51 46 33 
                    42 23 43 31 52 23 42  0 33 15 37 33 33 31 37 
                    51 23 41 62 21 55 23 33  0 29 62 46 29 51 11 
                    55 31 29 42 46 31 31 15 29  0 51 21 41 23 37 
                    29 41 79 21 82 33 77 37 62 51  0 65 42 59 61 
                    74 51 21 51 58 37 37 33 46 21 65  0 61 11 55 
                    23 11 64 51 46 51 51 33 29 41 42 61  0 62 23 
                    72 52 31 43 65 29 46 31 51 23 59 11 62  0 59 
                    46 21 51 64 23 59 33 37 11 37 61 55 23 59  0
                    '''.split()
                )
            )
        )[(i-1) * 15 + (j-1)]
    )

In [22]:
from typing import List, Callable
from pulp import LpVariable, LpProblem, LpConstraint, LpMinimize

def solve_tsp_with_lp(num_vertices: int, distance_function: Callable[[int, int], int]) -> List[int]:
    lp = LpProblem(name='TSP', sense=LpMinimize)
    indicator = list()
    for i in range(0, num_vertices):
        indicator.append([0]*num_vertices)
        for j in range(0, num_vertices):
            indicator[i][j] = LpVariable(name='from {} to {}'.format(i, j), cat='Binary')
    
    MTZ = [0] * num_vertices
    for i in range(0, num_vertices):
        MTZ[i] = LpVariable(name='MTZ {}'.format(i), lowBound=0, cat='Integer')        
    
    lp.objective = sum([distance_function(i + 1, j + 1) * indicator[i][j] for i in range(num_vertices) for j in range(num_vertices)])
    for i in range(0, num_vertices):
        for j in range(1, num_vertices):
            if i != j:
                lp += ((MTZ[i] - MTZ[j] + (num_vertices) * indicator[i][j]) <= (num_vertices - 1))
    
    for i in range(num_vertices):
        lp += (sum([indicator[j][i] for j in range(num_vertices)]) == 1)
        lp += (sum([indicator[i][j] for j in range(num_vertices)]) == 1)
        lp += (indicator[i][i] == False)
        
    lp_status = lp.solve()
    ans = [1]
    for i in range(num_vertices - 1):
        t = ans[-1] - 1
        for j in range(num_vertices):
            if indicator[t][j].value() == 1:
                ans.append(j + 1)
                break
    #for i in range(num_vertices):
    #    print()
    #    for j in range(num_vertices):
    #        print(indicator[i][j].value(), end=' ')
    return ans

print(' ⟶ '.join(map(str, solve_tsp_with_lp(15, dist15))))

1 ⟶ 11 ⟶ 4 ⟶ 6 ⟶ 8 ⟶ 10 ⟶ 14 ⟶ 12 ⟶ 3 ⟶ 7 ⟶ 5 ⟶ 9 ⟶ 15 ⟶ 2 ⟶ 13


## Как использовать библиотеку PuLP

Допустим, Вам нужно решить задачу СЦЛП: \begin{array}{lll}
2x+4y+3z & \to & \max,\\
x,y,z &\ge& 0,\\
x,y&\in& \mathbb{Z},\\
5x+2y&\le& 10,\\
7x+y &\le& 9,\\
3x+8z &\le& 11.
\end{array}

Ниже приведён достаточно само-описательный код программы, которая решает задачу с помощью библиотеки PuLP

In [14]:
from pulp import LpVariable, LpProblem, LpConstraint, LpMaximize

lp = LpProblem(name='My first linear program in PuLP', sense=LpMaximize)

x = LpVariable(name='x', lowBound=0, upBound=None, cat='Integer')
y = LpVariable(name='y', lowBound=0, upBound=None, cat='Integer')
z = LpVariable(name='z', lowBound=0, upBound=None, cat='Continuous')

lp.objective = 2*x + 4*y + 3*z

lp += (5*x + 2*y <= 10)
lp += (7*x + y <= 9)
lp += (3*x + 8*z <= 11)

lp_status = lp.solve()

print(str(lp_status))
print(
    x.value(), 
    y.value(), 
    z.value()
)

1
0.0 5.0 1.375
